<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">INFO</h1>

In this script, we create a class that will structure the unit tests for the `Dijkstra` player. \
We choose to use the `unittest` library. \
Then, we run them to ensure that all methods developed work as expected.

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">IMPORTS</h1>

In [ ]:
# External imports
from typing import *
from typing_extensions import *
from numbers import *
import unittest
import sys
import os
import random

# Add needed directories to the path
sys.path.append(os.path.join("..", "players"))

# PyRat imports
from GreedyEachTurn_opt import GreedyEachTurn
from pyrat import BigHolesRandomMaze, MazeFromDict

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">DEFINE THE TESTS</h1>

The `unittest` library requires the creation of a class that extends `unittest.TestCase`. \
For each method to test, we need to define a method in the test class. \
Each of these test methods should call the tested method with various inputs to check that produced outputs match expected ones.

In [ ]:
class GreedyEachTurnTests (unittest.TestCase):

    """
        This class tests the methods of the GreedyEachTurn class.
        For each method, we test it with a few different configurations.
    """

    #############################################################################################################################################
    #                                                                 UNIT TESTS                                                                #
    #############################################################################################################################################

    def test_traversal_random ( self: Self
                              ) ->    None:

        """
            This method tests the 'traversal' method.
            We are going to check the following:
            - Outputs are of the expected types.
            - All vertices are visited.
            - The routing table is a tree, with root corresponding to the start of the traversal.
            - The found distances are correct, i.e., strictly positive and increasing, except for the start vertex which should be 0.
            Note that we cannot test that the output is unique, as there are multiple valid outputs, depending on how vertices are explored.
            We will test the method on several random graphs to make sure it is working properly.
            Random graphs will be generated using the PyRat class used to create mazes in games.
            There are several such classes, but we will use the BigHolesRandomMaze class.
            To test that we indeed find the shortest path, we also test the method on some controlled examples in a separate test function.
            In:
                * self: Reference to the current object.
            Out:
                * None.
        """

        # Constants
        NB_GRAPHS = 10
        WIDTHS = [2, 30]
        HEIGHTS = [2, 30]
        CELL_PERCENTAGES = [20.0, 100.0]
        WALL_PERCENTAGES = [20.0, 100.0]
        MUD_PERCENTAGE = 10.0

        # Test on several graphs
        for i in range(NB_GRAPHS):
            
            # Instantiate the player
            player = GreedyEachTurn()

            # Generate a random maze
            # We use a fixed random seed for reproducibility of tests
            rng = random.Random(i)
            maze = BigHolesRandomMaze(width = rng.randint(WIDTHS[0], WIDTHS[1]),
                                      height = rng.randint(HEIGHTS[0], HEIGHTS[1]),
                                      cell_percentage = rng.uniform(CELL_PERCENTAGES[0], CELL_PERCENTAGES[1]),
                                      wall_percentage = rng.uniform(WALL_PERCENTAGES[0], WALL_PERCENTAGES[1]),
                                      mud_percentage = MUD_PERCENTAGE,
                                      mud_range=(4, 9),
                                      random_seed = i)

        

            # Choose a random starting vertex
            start_vertex = rng.choice(maze.vertices)

            # Perform the traversal
            distances, routing_table = player.traversal(maze, start_vertex)

            # Check the output type for distances
            # It should be a dictionary with integer keys and integer values
            self.assertTrue(isinstance(distances, Dict))
            self.assertTrue(all(isinstance(k, Integral) for k in distances.keys()))
            self.assertTrue(all(isinstance(v, Integral) for v in distances.values()))
            
            # Check the output type for the routing table
            # It should be a dictionary with integer keys and integer or None values
            self.assertTrue(isinstance(routing_table, Dict))
            self.assertTrue(all(isinstance(k, Integral) for k in routing_table.keys()))
            self.assertTrue(all(isinstance(v, (type(None), Integral)) for v in routing_table.values()))

            # All vertices should be visited
            # Equivalently, the distances should have the same keys as the maze vertices
            self.assertEqual(sorted(set(distances.keys())), sorted(maze.vertices))

            # Check that the start vertex is the only tree root
            self.assertEqual(routing_table[start_vertex], None)
            self.assertTrue(all(routing_table[v] is not None for v in routing_table.keys() if v != start_vertex))
            self.assertEqual(distances[start_vertex], 0)
            self.assertTrue(all(distances[v] > 0 for v in distances.keys() if v != start_vertex))
            
            # We check that the routing table is a tree
            # Also we check that associated distances are decreasing as we go to the root
            for vertex in routing_table:

                # We check that the parent is closer to the root
                if routing_table[vertex] is not None:
                    self.assertTrue(distances[routing_table[vertex]] < distances[vertex])
                
                # We ckeck that we can reach the root from any vertex
                path = [vertex]
                while routing_table[path[-1]] is not None:
                    self.assertTrue(routing_table[path[-1]] not in path)
                    path.append(routing_table[path[-1]])
    
    #############################################################################################################################################

    def test_traversal_fixed ( self: Self
                             ) ->    None:

        """
            This method complements the previous tests by testing the 'traversal' method on some controlled examples.
            To do so, we will use some mazes for which there is a unique routing table.
            We do not test the output types again, as it has already been done in the previous test function.
            In:
                * self: Reference to the current object.
            Out:
                * None.
        """

        # Constants
        TESTS = [{"inputs": {"maze": MazeFromDict({0: {15: 1}, 1: {16: 1}, 6: {7: 8, 21: 8}, 7: {6: 8}, 8: {9: 1, 23: 1}, 9: {8: 1, 10: 1}, 10: {9: 1, 25: 1}, 11: {12: 1, 26: 1}, 12: {11: 1, 13: 9}, 13: {14: 1, 12: 9}, 14: {29: 1, 13: 1}, 15: {0: 1, 16: 1, 30: 1}, 16: {1: 1, 15: 1, 31: 1}, 21: {22: 1, 36: 1, 6: 8}, 22: {21: 1, 23: 1, 37: 7}, 23: {8: 1, 22: 1, 38: 1}, 25: {26: 1, 10: 1, 40: 5}, 26: {11: 1, 25: 1, 41: 1}, 28: {29: 1}, 29: {14: 1, 28: 1, 44: 5}, 30: {15: 1, 31: 1, 45: 8}, 31: {32: 1, 16: 1, 30: 1, 46: 6}, 32: {31: 1}, 34: {35: 1, 49: 1}, 35: {34: 1, 36: 1, 50: 8}, 36: {21: 1, 35: 1}, 37: {22: 7, 52: 4}, 38: {23: 1, 53: 1}, 40: {25: 5, 41: 4}, 41: {26: 1, 40: 4, 56: 9}, 44: {59: 4, 29: 5}, 45: {46: 1, 30: 8, 60: 8}, 46: {45: 1, 47: 1, 61: 1, 31: 6}, 47: {46: 1, 48: 1, 62: 6}, 48: {47: 1, 49: 1}, 49: {48: 1, 34: 1, 64: 9}, 50: {65: 4, 35: 8}, 52: {53: 5, 67: 4, 37: 4}, 53: {38: 1, 68: 1, 52: 5}, 55: {56: 1, 70: 1}, 56: {55: 1, 57: 6, 41: 9}, 57: {58: 1, 72: 1, 56: 6}, 58: {57: 1, 59: 1}, 59: {58: 1, 44: 4}, 60: {75: 1, 45: 8}, 61: {46: 1, 76: 1}, 62: {63: 1, 77: 1, 47: 6}, 63: {62: 1, 78: 1}, 64: {65: 1, 79: 1, 49: 9}, 65: {64: 1, 50: 4, 80: 6}, 67: {82: 8, 52: 4, 68: 8}, 68: {53: 1, 83: 1, 67: 8}, 70: {55: 1, 71: 1, 85: 1}, 71: {70: 1}, 72: {57: 1, 87: 1, 73: 1}, 73: {88: 1, 72: 1, 74: 4}, 74: {73: 4, 89: 4}, 75: {90: 1, 60: 1, 76: 5}, 76: {77: 1, 61: 1, 91: 1, 75: 5}, 77: {76: 1, 78: 1, 62: 1, 92: 5}, 78: {63: 1, 77: 1, 79: 9}, 79: {64: 1, 94: 1, 78: 9}, 80: {81: 1, 65: 6}, 81: {80: 1, 82: 1}, 82: {81: 1, 83: 1, 97: 1, 67: 8}, 83: {68: 1, 82: 1, 98: 1}, 85: {70: 1, 86: 1, 100: 1}, 86: {85: 1, 87: 1, 101: 5}, 87: {72: 1, 86: 1, 88: 1}, 88: {73: 1, 87: 1, 89: 1}, 89: {88: 1, 104: 1, 74: 4}, 90: {91: 1, 75: 1}, 91: {90: 1, 106: 1, 76: 1}, 92: {107: 8, 77: 5, 93: 5}, 93: {94: 1, 92: 5, 108: 9}, 94: {79: 1, 93: 1}, 97: {112: 1, 82: 1, 98: 1}, 98: {83: 1, 99: 1, 97: 1}, 99: {98: 1, 114: 1}, 100: {85: 1, 115: 6, 101: 9}, 101: {116: 1, 102: 6, 86: 5, 100: 9}, 102: {101: 6}, 103: {118: 1}, 104: {89: 1}, 106: {91: 1, 107: 1}, 107: {106: 1, 108: 1, 92: 8}, 108: {107: 1, 123: 1, 93: 9}, 112: {97: 1, 127: 1, 113: 4}, 113: {114: 1, 128: 1, 112: 4}, 114: {99: 1, 113: 1, 129: 1, 115: 6}, 115: {116: 1, 130: 1, 114: 6, 100: 6}, 116: {101: 1, 115: 1, 117: 1}, 117: {116: 1, 118: 1}, 118: {103: 1, 117: 1}, 120: {121: 1}, 121: {120: 1, 136: 1, 122: 1}, 122: {121: 1, 123: 9, 137: 9}, 123: {108: 1, 138: 1, 122: 9}, 126: {127: 1}, 127: {112: 1, 126: 1}, 128: {113: 1}, 129: {114: 1, 144: 1, 130: 8}, 130: {115: 1, 129: 8}, 135: {150: 1}, 136: {121: 1, 151: 7, 137: 4}, 137: {138: 1, 152: 1, 122: 9, 136: 4}, 138: {123: 1, 137: 1, 139: 1}, 139: {138: 1, 140: 1, 154: 9}, 140: {139: 1, 155: 1}, 141: {156: 1}, 142: {157: 1}, 143: {158: 6}, 144: {129: 1, 159: 1}, 149: {164: 1}, 150: {135: 1, 151: 1, 165: 1}, 151: {150: 1, 136: 7}, 152: {137: 1}, 153: {168: 1, 154: 1}, 154: {153: 1, 139: 9, 155: 8}, 155: {140: 1, 156: 1, 170: 1, 154: 8}, 156: {141: 1, 155: 1, 157: 1}, 157: {142: 1, 156: 1, 158: 1, 172: 1}, 158: {157: 1, 173: 1, 159: 5, 143: 6}, 159: {144: 1, 174: 1, 158: 5, 160: 6}, 160: {159: 6}, 163: {164: 1, 178: 6}, 164: {149: 1, 163: 1, 179: 1}, 165: {150: 1, 166: 1, 180: 1}, 166: {165: 1, 181: 1}, 167: {168: 1}, 168: {153: 1, 167: 1, 183: 1, 169: 6}, 169: {170: 1, 184: 6, 168: 6}, 170: {169: 1, 171: 1, 155: 1}, 171: {170: 1, 186: 1, 172: 6}, 172: {157: 1, 171: 6}, 173: {158: 1, 188: 7, 174: 8}, 174: {159: 1, 175: 1, 189: 1, 173: 8}, 175: {174: 1}, 177: {178: 1, 192: 1}, 178: {177: 1, 179: 1, 193: 1, 163: 6}, 179: {164: 1, 178: 1, 194: 4}, 180: {181: 1, 165: 1}, 181: {180: 1, 166: 1, 182: 5}, 182: {183: 1, 181: 5}, 183: {168: 1, 182: 1}, 184: {185: 1, 169: 6}, 185: {184: 1, 186: 7}, 186: {187: 1, 171: 1, 185: 7}, 187: {186: 1, 188: 6}, 188: {189: 9, 173: 7, 187: 6}, 189: {174: 1, 190: 1, 188: 9}, 190: {189: 1, 191: 1}, 191: {190: 1, 192: 1}, 192: {177: 1, 191: 1}, 193: {178: 1, 194: 1}, 194: {193: 1, 179: 4}}),
                             "start": 97},
                  "outputs": {"routing_table": {97: None, 112: 97, 82: 97, 98: 97, 81: 82, 83: 82, 67: 82, 99: 98, 127: 112, 113: 114, 80: 81, 68: 83, 114: 99, 126: 127, 53: 68, 65: 80, 129: 114, 115: 114, 38: 53, 52: 53, 128: 113, 144: 129, 130: 115, 23: 38, 159: 144, 8: 23, 22: 23, 174: 159, 158: 159, 160: 159, 9: 8, 21: 22, 37: 52, 175: 174, 189: 174, 173: 158, 10: 9, 36: 21, 6: 21, 190: 189, 188: 189, 25: 10, 35: 36, 64: 65, 50: 65, 116: 115, 100: 115, 191: 190, 26: 25, 40: 25, 34: 35, 79: 64, 49: 34, 101: 116, 117: 116, 192: 191, 11: 26, 41: 26, 94: 79, 78: 77, 102: 101, 86: 101, 118: 117, 157: 158, 143: 158, 177: 192, 12: 11, 56: 55, 48: 49, 93: 94, 103: 118, 142: 157, 156: 157, 172: 157, 178: 177, 13: 12, 47: 48, 92: 93, 108: 123, 141: 156, 155: 156, 171: 170, 179: 178, 193: 178, 163: 164, 46: 47, 62: 77, 140: 155, 170: 155, 154: 155, 164: 179, 194: 193, 45: 46, 61: 46, 31: 46, 85: 100, 139: 140, 149: 164, 169: 170, 7: 6, 30: 31, 60: 75, 76: 61, 70: 85, 87: 86, 138: 139, 184: 169, 168: 169, 186: 171, 55: 70, 71: 70, 77: 76, 91: 76, 75: 90, 72: 87, 88: 87, 123: 138, 137: 138, 187: 186, 185: 184, 57: 72, 73: 72, 89: 88, 90: 91, 106: 91, 107: 106, 122: 121, 152: 137, 136: 137, 58: 57, 63: 62, 74: 73, 104: 89, 59: 58, 32: 31, 16: 31, 44: 59, 14: 13, 1: 16, 15: 16, 121: 136, 151: 136, 153: 154, 167: 168, 183: 168, 29: 14, 0: 15, 120: 121, 182: 183, 28: 29, 181: 182, 150: 151, 180: 181, 166: 181, 135: 150,165: 150},
                              "distances": {97: 0, 112: 1, 82: 1, 98: 1, 81: 2, 83: 2, 67: 9, 99: 2, 127: 2, 113: 4, 80: 3, 68: 3, 114: 3, 126: 3, 53: 4, 65: 9, 129: 4, 115: 9, 38: 5, 52: 9, 128: 5, 144: 5, 130: 10, 23: 6, 159: 6, 8: 7, 22: 7, 174: 7, 158: 11, 160: 12, 9: 8, 21: 8, 37: 13, 175: 8, 189: 8, 173: 12, 10: 9, 36: 9, 6: 16, 190: 9, 188: 17, 25: 10, 35: 10, 64: 10, 50: 13, 116: 10, 100: 15, 191: 10, 26: 11, 40: 15, 34: 11, 79: 11, 49: 12, 101: 11, 117: 11, 192: 11, 11: 12, 41: 12, 94: 12, 78: 19, 102: 17, 86: 16, 118: 12, 157: 12, 143: 17, 177: 12, 12: 13, 56: 19, 48: 13, 93: 13, 103: 13, 142: 13, 156: 13, 172: 13, 178: 13, 13: 22, 47: 14, 92: 18, 108: 19, 141: 14, 155: 14, 171: 16, 179: 14, 193: 14, 163: 16, 46: 15, 62: 19, 140: 15, 170: 15, 154: 22, 164: 15, 194: 15, 45: 16, 61: 16, 31: 21, 85: 16, 139: 16, 149: 16, 169: 16, 7: 24, 30: 22, 60: 21, 76: 17, 70: 17, 87: 17, 138: 17, 184: 22, 168: 22, 186: 17, 55: 18, 71: 18, 77: 18, 91: 18, 75: 20, 72: 18, 88: 18, 123: 18, 137: 18, 187: 18, 185: 23, 57: 19, 73: 19, 89: 19, 90: 19, 106: 19, 107: 20, 122: 24, 152: 19, 136: 22, 58: 20, 63: 20, 74: 23, 104: 20, 59: 21, 32: 22, 16: 22, 44: 25, 14: 23, 1: 23, 15: 23, 121: 23, 151: 29, 153: 23, 167: 23, 183: 23, 29: 24, 0: 24, 120: 24, 182: 24, 28: 25, 181: 29, 150: 30, 180: 30, 166: 30, 135:31,165:31}}},
                 {"inputs": {"maze": MazeFromDict({0: {15: 1, 1: 1}, 1: {2: 1, 0: 1}, 2: {1: 1, 3: 1}, 3: {2: 1, 4: 1, 18: 4}, 4: {3: 1, 19: 1, 5: 1}, 5: {20: 1, 4: 1, 6: 1}, 6: {21: 1, 5: 1}, 7: {8: 1}, 8: {7: 1, 9: 7}, 9: {24: 1, 10: 7, 8: 7}, 10: {11: 1, 9: 7}, 11: {10: 1, 12: 1}, 12: {11: 1, 13: 1}, 13: {12: 1}, 14: {29: 1}, 15: {0: 1, 16: 1}, 16: {15: 1, 31: 1}, 17: {32: 1, 18: 1}, 18: {19: 1, 33: 1, 17: 1, 3: 4}, 19: {4: 1, 18: 1, 34: 1}, 20: {5: 1, 35: 1}, 21: {6: 1, 36: 1, 22: 1}, 22: {37: 1, 21: 1}, 24: {9: 1, 39: 1}, 29: {14: 1, 44: 1}, 30: {31: 1}, 31: {16: 1, 30: 1, 46: 1}, 32: {17: 1, 33: 7}, 33: {34: 1, 18: 1, 32: 7}, 34: {19: 1, 33: 1, 49: 1, 35: 5}, 35: {20: 1, 50: 1, 34: 5}, 36: {21: 1, 37: 1}, 37: {22: 1, 36: 1, 52: 1, 38: 1}, 38: {39: 1, 37: 1}, 39: {24: 1, 38: 1, 54: 5}, 41: {56: 1}, 42: {57: 1}, 44: {29: 1, 59: 5}, 45: {46: 1}, 46: {31: 1, 45: 1, 61: 1, 47: 1}, 47: {46: 1, 48: 1, 62: 4}, 48: {47: 1, 49: 5}, 49: {64: 1, 34: 1, 48: 5}, 50: {35: 1, 65: 1}, 52: {37: 1, 67: 1, 53: 1}, 53: {68: 1, 52: 1, 54: 5}, 54: {69: 1, 55: 7, 53: 5, 39: 5}, 55: {56: 1, 54: 7, 70: 6}, 56: {41: 1, 55: 1, 71: 1, 57: 6}, 57: {42: 1, 58: 1, 72: 1, 56: 6}, 58: {57: 1, 59: 1}, 59: {58: 1, 44: 5, 74: 6}, 60: {75: 1}, 61: {46: 1, 76: 4}, 62: {63: 1, 47: 4}, 63: {62: 1, 78: 8}, 64: {49: 1, 65: 1, 79: 8}, 65: {50: 1, 64: 1, 66: 1}, 66: {65: 1, 67: 1, 81: 1}, 67: {52: 1, 66: 1}, 68: {83: 1, 53: 1}, 69: {54: 1, 84: 1}, 70: {85: 1, 55: 6}, 71: {86: 1, 56: 1}, 72: {57: 1, 87: 9}, 74: {59: 6}, 75: {60: 1, 76: 1}, 76: {75: 1, 91: 1, 77: 1, 61: 4}, 77: {78: 1, 76: 1, 92: 1}, 78: {77: 1, 93: 1, 63: 8}, 79: {80: 1, 94: 1, 64: 8}, 80: {79: 1, 81: 6}, 81: {66: 1, 96: 1, 82: 1, 80: 6}, 82: {83: 1, 81: 1, 97: 1}, 83: {68: 1, 82: 1, 98: 1}, 84: {69: 1, 85: 1, 99: 4}, 85: {84: 1, 100: 1, 70: 1, 86: 4}, 86: {71: 1, 101: 6, 85: 4}, 87: {102: 1, 72: 9}, 90: {105: 1, 91: 5}, 91: {76: 1, 92: 1, 106: 1, 90: 5}, 92: {91: 1, 107: 1, 93: 1, 77: 1}, 93: {78: 1, 108: 1, 92: 1, 94: 6}, 94: {79: 1, 93: 6, 95: 6}, 95: {110: 1, 94: 6, 96: 7}, 96: {81: 1, 111: 1, 95: 7}, 97: {112: 1, 82: 1}, 98: {83: 1, 99: 1, 113: 1}, 99: {98: 1, 100: 1, 114: 1, 84: 4}, 100: {99: 1, 85: 1}, 101: {116: 1, 86: 6, 102: 8}, 102: {87: 1, 103: 1, 101: 8}, 105: {106: 1, 90: 1, 120: 8}, 106: {91: 1, 105: 1, 107: 8}, 107: {122: 1, 92: 1, 108: 1, 106: 8}, 108: {93: 1, 123: 1, 107: 1, 109: 9}, 109: {110: 4, 108: 9}, 110: {95: 1, 125: 1, 111: 1, 109: 4}, 111: {96: 1, 126: 1, 110: 1}, 112: {97: 1, 113: 1, 127: 1}, 113: {98: 1, 112: 1, 128: 1, 114: 5}, 114: {99: 1, 129: 1, 113: 5, 115: 5}, 115: {116: 1, 114: 5}, 116: {101: 1, 115: 1}, 120: {135: 4, 105: 8}, 121: {122: 4}, 122: {107: 1, 123: 1, 121: 4}, 123: {108: 1, 122: 1, 138: 1, 124: 6}, 124: {125: 1, 123: 6}, 125: {110: 1, 124: 1, 126: 8, 140: 7}, 126: {111: 1, 125: 8, 127: 5}, 127: {112: 1, 126: 5}, 128: {113: 1, 129: 4}, 129: {114: 1, 128: 4}, 135: {150: 1, 120: 4}, 137: {152: 1, 138: 5}, 138: {123: 1, 139: 1, 137: 5}, 139: {140: 1, 154: 1, 138: 1}, 140: {139: 1, 141: 1, 125: 7}, 141: {140: 1, 142: 1, 156: 1}, 142: {141: 1, 143: 1, 157: 1}, 143: {142: 1}, 150: {135: 1, 165: 1, 151: 7}, 151: {166: 1, 152: 7, 150: 7}, 152: {137: 1, 151: 7}, 154: {139: 1, 169: 1}, 156: {157: 1, 141: 1, 171: 5}, 157: {142: 1, 156: 1, 158: 8, 172: 7}, 158: {157: 8}, 160: {175: 1}, 162: {177: 1}, 165: {150: 1, 180: 8}, 166: {151: 1}, 169: {154: 1, 170: 5}, 170: {169: 5}, 171: {156: 5, 172: 4}, 172: {173: 1, 187: 1, 157: 7, 171: 4}, 173: {172: 1, 188: 1, 174: 7}, 174: {173: 7, 175: 5}, 175: {160: 1, 190: 4, 174: 5, 176: 5}, 176: {177: 1, 191: 6, 175: 5}, 177: {162: 1, 176: 1, 192: 1}, 180: {165: 8, 181: 5}, 181: {182: 7, 180: 5}, 182: {181: 7}, 184: {185: 7}, 185: {186: 1, 184: 7}, 186: {185: 1, 187: 1}, 187: {172: 1, 186: 1, 188: 1}, 188: {173: 1, 187: 1}, 190: {191: 1, 175: 4}, 191: {190: 1, 176: 6}, 192: {177: 1, 193: 9}, 193: {192: 9}, 103: {102: 1, 104: 1}, 104: {103: 1} }),
                             "start": 97},
                  "outputs": {"routing_table": {97: None, 112: 97, 82: 97, 83: 82, 81: 82, 113: 112, 127: 112, 66: 81, 96: 81, 80: 81, 68: 83, 98: 83, 128: 113, 114: 99, 126: 111, 65: 66, 67: 66, 53: 68, 111: 96, 95: 110, 99: 98, 129: 114, 52: 53, 54: 53, 50: 65, 64: 65, 100: 99, 84: 85, 110: 111, 35: 50, 37: 52, 49: 64, 79: 80, 85: 100, 125: 110, 109: 110, 115: 114, 20: 35, 34: 49, 22: 37, 36: 37, 38: 37, 48: 49, 70: 85, 86: 85, 94: 79, 124: 125, 140: 125, 5: 20, 21: 22, 19: 34, 33: 34, 39: 38, 55: 70, 69: 84, 123: 124, 4: 5, 6: 5, 18: 19, 32: 17, 24: 39, 3: 4, 17: 18, 9: 24, 108: 123, 2: 3, 10: 9, 8: 9, 71: 86, 101: 116, 93: 108, 116: 115, 1: 2, 47: 48, 56: 71, 0: 1, 46: 47, 62: 47, 41: 56, 57: 56, 102: 101, 15: 0, 31: 46, 45: 46, 61: 46, 122: 123, 138: 123, 139: 140, 141: 140, 16: 15, 30: 31, 76: 61, 107: 108, 121: 122, 137: 138, 154: 139, 142: 141, 156: 141, 78: 93, 92: 93, 106: 91, 143: 142, 157: 142, 169: 154, 171: 156, 63: 62, 77: 78, 91: 92, 158: 157, 172: 157, 170: 169, 7: 8, 11: 10, 90: 105, 12: 11, 42: 57, 58: 57, 72: 57, 75: 76, 105: 106, 13: 12, 59: 58, 87: 102, 60: 75, 120: 105, 152: 137, 44: 59, 74: 59, 103: 102, 151: 152, 104: 103, 173: 172, 187: 172, 188: 173, 174: 173, 186: 187, 29: 44, 185: 186, 14: 29, 184: 185, 135: 120, 166: 151, 150: 135, 175: 174, 165: 150, 180: 165, 160: 175, 190: 175, 176: 175, 191: 190, 177: 176, 181: 180, 162: 177, 192: 177, 193:192,182:181},
                              "distances": {97: 0, 112: 1, 82: 1, 83: 2, 81: 2, 113: 2, 127: 2, 66: 3, 96: 3, 80: 8, 68: 3, 98: 3, 128: 3, 114: 5, 126: 5, 65: 4, 67: 4, 53: 4, 111: 4, 95: 6, 99: 4, 129: 6, 52: 5, 54: 9, 50: 5, 64: 5, 100: 5, 84: 7, 110: 5, 35: 6, 37: 6, 49: 6, 79: 9, 85: 6, 125: 6, 109: 9, 115: 10, 20: 7, 34: 7, 22: 7, 36: 7, 38: 7, 48: 11, 70: 7, 86: 10, 94: 10, 124: 7, 140: 13, 5: 8, 21: 8, 19: 8, 33: 8, 39: 8, 55: 13, 69: 8, 123: 13, 4: 9, 6: 9, 18: 9, 32: 11, 24: 9, 3: 10, 17: 10, 9: 10, 108: 14, 2: 11, 10: 17, 8: 17, 71: 11, 101: 12, 93: 15, 116: 11, 1: 12, 47: 12, 56: 12, 0: 13, 46: 13, 62: 16, 41: 13, 57: 18, 102: 20, 15: 14, 31: 14, 45: 14, 61: 14, 122: 14, 138: 14, 139: 14, 141: 14, 16: 15, 30: 15, 76: 18, 107: 15, 121: 18, 137: 19, 154: 15, 142: 15, 156: 15, 78: 16, 92: 16, 106: 18, 143: 16, 157: 16, 169: 16, 171: 20, 63: 17, 77: 17, 91: 17, 158: 24, 172: 23, 170: 21, 7: 18, 11: 18, 90: 20, 12: 19, 42: 19, 58: 19, 72: 19, 75: 19, 105: 19, 13: 20, 59: 20, 87: 21, 60: 20, 120: 27, 152: 20, 44: 25, 74: 26, 103: 21, 151: 27, 104: 22, 173: 24, 187: 24, 188: 25, 174: 31, 186: 25, 29: 26, 185: 26, 14: 27, 184: 33, 135: 31, 166: 28, 150: 32, 175: 36, 165: 33, 180: 41, 160: 37, 190: 40, 176: 41, 191: 41, 177: 42, 181: 46, 162: 43, 192: 43, 193:52,182:53}}}]
                
        # Test on several controlled examples
        for i in range(len(TESTS)):
            
            # Instantiate the player
            player = GreedyEachTurn()

            # Get the input and output data
            maze = TESTS[i]["inputs"]["maze"]
            start = TESTS[i]["inputs"]["start"]
            target_routing_table = TESTS[i]["outputs"]["routing_table"]
            target_distances = TESTS[i]["outputs"]["distances"]

            # Perform the traversal
            distances, routing_table = player.traversal(maze, start)

            # Check that outputs match the expected ones
            self.assertEqual(sorted(routing_table), sorted(target_routing_table))
            self.assertEqual(sorted(distances), sorted(target_distances))

    #############################################################################################################################################

    def test_find_route ( self: Self,
                        ) ->    None:
        
        """
            This method tests the 'find_route' method.
            Here, we want to make sure that the output corresponds indeed to a route from the start to the end.
            We are going to check the following:
            - Outputs are of the expected types.
            - The route is a valid path from the start to the end.
            The method will be tested on some controlled examples, where we can easily check the validity of the output.
            In:
                * self: Reference to the current object.
            Out:
                * None.
        """

        # Constants
        INPUTS = [{"routing_table": {0: None, 1: 0, 2: 1, 3: 2, 4: 3},
                       "start": 0,
                       "end": 4},
                  {"routing_table": {0: 2, 1: None, 2: 1, 3: 0, 4: 0, 5: 4, 6: 1},
                       "start": 1,
                       "end": 6}]

        # Test on several controlled examples
        for i in range(len(INPUTS)):
            
            # Instantiate the player
            player = GreedyEachTurn()

            # Get the input data
            routing_table = INPUTS[i]["routing_table"]
            start = INPUTS[i]["start"]
            end = INPUTS[i]["end"]

            # Find the route
            route = player.find_route(routing_table, start, end)

            # Check the output type
            # It should be a list of integers
            self.assertTrue(isinstance(route, List))
            self.assertTrue(all(isinstance(v, Integral) for v in route))

            # Check that the route is a valid path from the start to the end
            self.assertEqual(route[0], start)
            self.assertEqual(route[-1], end)
            self.assertTrue(all(routing_table[route[j + 1]] == route[j] for j in range(len(route) - 1)))
    def test_neighbors_sorted(self):
        meta_graph = {
            0: {1: (1, [0, 1]), 2: (2, [0, 2]), 3: (3, [0, 3])},
            1: {0: (1, [1, 0]), 2: (4, [1, 2]), 3: (6, [1, 3])},
            2: {0: (2, [2, 0]), 1: (4, [2, 1]), 3: (5, [2, 3])},
            3: {0: (3, [3, 0]), 1: (6, [3,1]), 2: (5, [3,2])}
        }
        player = GreedyEachTurn()
        player.neighbors_sorted(0, [1, 2, 3], meta_graph)

        self.assertEqual(player.the_nearest , {0:[(1, 1), (2, 2), (3, 3)],1:[(0,1),(2,4),(3,6)],2:[(0,2),(1,4),(3,5)],3:[(0,3),(2,5),(1,6)]})
    def test_meta_graph(self):
        maze =MazeFromDict({0: {6: 1}, 1: {7: 1}, 2: {8: 1}, 3: {4: 1}, 4: {3: 1, 5: 1, 10: 1}, 5: {4: 1}, 6: {0: 1, 12: 1, 7: 1}, 
                            7: {1: 1, 8: 1, 6: 1, 13: 5}, 8: {2: 1, 7: 1, 14: 8}, 10: {4: 1, 11: 1, 16: 8}, 11: {10: 1, 17: 1}, 
                            12: {6: 1, 13: 1}, 13: {12: 1, 14: 1, 19: 1, 7: 5}, 14: {13: 1, 8: 8}, 16: {22: 1, 10: 8, 17: 4}, 
                            17: {23: 1, 11: 1, 16: 4}, 19: {13: 1, 25: 5}, 22: {16: 1, 23: 1, 28: 1}, 23: {22: 1, 17: 1}, 25: {26: 1, 19: 5}, 
                            26: {25: 1, 27: 1}, 27: {26: 1, 28: 1}, 28: {22: 1, 27: 1, 29: 1}, 29: {28: 1}})
        player = GreedyEachTurn()
        source = 0
        cheese = [5,10,22]
        graph = player.meta_graph(maze, source, cheese)
        OUTPUT={5: {0: (19, [5, 4, 10, 11, 17, 23, 22, 28, 27, 26, 25, 19, 13, 12, 6, 0]), 22: (6, [5, 4, 10, 11, 17, 23, 22]), 10: (2, [5, 4, 10])}, 
                10: {0: (17, [10, 11, 17, 23, 22, 28, 27, 26, 25, 19, 13, 12, 6, 0]), 22: (4, [10, 11, 17, 23, 22]), 5: (2, [10, 4, 5])}, 
                22: {0: (13, [22, 28, 27, 26, 25, 19, 13, 12, 6, 0]), 5: (6, [22, 23, 17, 11, 10, 4, 5]), 10: (4, [22, 23, 17, 11, 10])}, 
                0: {5: (19, [0, 6, 12, 13, 19, 25, 26, 27, 28, 22, 23, 17, 11, 10, 4, 5]), 10: (17, [0, 6, 12, 13, 19, 25, 26, 27, 28, 22, 23, 17, 11, 10]), 22: (13, [0, 6, 12, 13, 19, 25, 26, 27, 28, 22])}}
        self.assertEqual(player.meta_graph(maze,source,cheese),OUTPUT)


<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">RUN THE TESTS</h1>
           
When calling `unittest.main()`, all methods in the test class above will be run.

In [3]:
# Run all tests
_ = unittest.main(argv=[""], verbosity=2, exit=False)

test_find_route (__main__.DijkstraTests.test_find_route)
This method tests the 'find_route' method. ... ok
test_traversal_fixed (__main__.DijkstraTests.test_traversal_fixed)
This method complements the previous tests by testing the 'traversal' method on some controlled examples. ... ok
test_traversal_random (__main__.DijkstraTests.test_traversal_random)
This method tests the 'traversal' method. ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.472s

OK
